## SRJ

In [1]:
import pennylane as qml




In [2]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                print('Exc. dealing right now is', ash_excitation[i])
                print('The params that are going in', params[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
            elif len(ash_excitation[i]) == 2:
                print('Single Exc. dealing right now is', ash_excitation[i])
                print('Single exc params that are going in', params[i])
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        print('The current status of ash_excitation is', ash_excitation)
        print('Moving towards parameters')
        #params = np.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
        #print('The length of parameters is', params)
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
            print('Parameters are', params)
        else:
            params = np.append(params, 0.0) 
            print('Parameters before optimization are', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs_state, H, qubits









In [3]:
energies, params, ash_excitation, gs_state,H, qubits = adaptvqe(adapt_it = 1, e_th=1e-12)
print('Energies:', energies)


Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.332568888066893
The adapt iteration now is 0
The current excitation operator is a⁺(0) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(4) is 9.258181025983969e-08
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(6) is 0.0
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(5) is 9.258181027371748e-08
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(2) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(4) is 0.0
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(6) is 1.1698968021417455e-07
The current excitation operator is a⁺(3) a(5)
Print, if this is activated - HF 

/Users/s.poyyapakkam/miniconda3/envs/sri/lib/python3.12/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 0.0
The current excitation operator is a⁺(0) a⁺(1) a(6) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.28410686098083854
The current excitation operator is a⁺(0) a⁺(2) a(4) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.030544625693072067
The current excitation operator is a⁺(0) a⁺(3) a(4) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 0.0
The current excitation operator is a⁺(0) a⁺(3) a(4) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.3078190525120401
The current excitation operator is a⁺(0) a⁺(3) a(5) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.277274426818968
The current excitation operator is a⁺(0) a⁺(3) a(6) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(0) a⁺(3) a

In [ ]:
paramss = np.array([0.1] * (2 * qubits), requires_grad=True)
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def k2(paramss):
    qml.StatePrep(gs_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss[i + qubits], wires=i)
    
    return qml.expval(H)

print('Params before optimizeation', paramss)

print(qml.draw(k2, max_length=200)(paramss))


optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

for n in range(450):
    paramss, energy = optimizer.step_and_cost(k2, paramss)
    print('The 1st set params after calculation :', paramss)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
print(qml.draw(k2, max_length=200)(paramss))

Params before optimizeation [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
0: ─╭|Ψ⟩──RY(0.10)─╭●──RY(0.10)─────────────────────────────────────────────────────────────┤ ╭<𝓗>
1: ─├|Ψ⟩──RY(0.10)─╰X─╭●─────────RY(0.10)───────────────────────────────────────────────────┤ ├<𝓗>
2: ─├|Ψ⟩──RY(0.10)────╰X────────╭●─────────RY(0.10)─────────────────────────────────────────┤ ├<𝓗>
3: ─├|Ψ⟩──RY(0.10)──────────────╰X────────╭●─────────RY(0.10)───────────────────────────────┤ ├<𝓗>
4: ─├|Ψ⟩──RY(0.10)────────────────────────╰X────────╭●─────────RY(0.10)─────────────────────┤ ├<𝓗>
5: ─├|Ψ⟩──RY(0.10)──────────────────────────────────╰X────────╭●─────────RY(0.10)───────────┤ ├<𝓗>
6: ─├|Ψ⟩──RY(0.10)────────────────────────────────────────────╰X────────╭●─────────RY(0.10)─┤ ├<𝓗>
7: ─╰|Ψ⟩──RY(0.10)──────────────────────────────────────────────────────╰X─────────RY(0.10)─┤ ╰<𝓗>
The 1st set params after calculation : [0.10745693 0.0931414  0.10382028 0.09836348 0.10924233 0.11056329
 0.10889

In [6]:
paramss

tensor([ 4.69653155e-06, -1.89247037e-07, -2.28874545e-03,
        -1.49877482e-09,  7.58459358e-02,  1.18150931e-04,
         1.57079601e+00,  1.62442177e+00, -2.28706205e-07,
         4.85807577e-06, -1.22518008e-06, -1.30044226e-07,
         3.14158402e+00, -1.18062338e-04,  1.57079653e+00,
        -1.57079480e+00], requires_grad=True)

In [9]:
dev3 = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev3)
def ostatehea(paramss):
    qml.StatePrep(gs_state, wires=range(qubits))
    
    # First RY layer (params[0-7])
    for i in range(qubits):
        qml.RY(paramss[i], wires=i)
    
    # CNOT gates
    for i in range(qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    # Second RY layer (params[8-15])
    for i in range(qubits):
        qml.RY(paramss[i + qubits], wires=i)
    
    return qml.state()
ostatehea(paramss)
print(qml.draw(ostatehea, max_length=200)(paramss))

0: ─╭|Ψ⟩──RY(0.00)──╭●──RY(-0.00)─────────────────────────────────────────────────────────────────┤  State
1: ─├|Ψ⟩──RY(-0.00)─╰X─╭●──────────RY(0.00)───────────────────────────────────────────────────────┤  State
2: ─├|Ψ⟩──RY(-0.00)────╰X─────────╭●─────────RY(-0.00)────────────────────────────────────────────┤  State
3: ─├|Ψ⟩──RY(-0.00)───────────────╰X────────╭●──────────RY(-0.00)─────────────────────────────────┤  State
4: ─├|Ψ⟩──RY(0.08)──────────────────────────╰X─────────╭●──────────RY(3.14)───────────────────────┤  State
5: ─├|Ψ⟩──RY(0.00)─────────────────────────────────────╰X─────────╭●─────────RY(-0.00)────────────┤  State
6: ─├|Ψ⟩──RY(1.57)────────────────────────────────────────────────╰X────────╭●──────────RY(1.57)──┤  State
7: ─╰|Ψ⟩──RY(1.62)──────────────────────────────────────────────────────────╰X──────────RY(-1.57)─┤  State
